In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
dataset = pd.read_csv("/kaggle/input/titanic/train.csv")
X = dataset.iloc[:, np.r_[2:3, 4:8]].values
y = dataset.iloc[:,1].values

In [3]:
dataset = pd.read_csv("/kaggle/input/titanic/test.csv")
X_test = dataset.iloc[:, np.r_[1:2, 3:7]].values
y_test = dataset.iloc[:,0].values

In [4]:
print(X)

[[3 'male' 22.0 1 0]
 [1 'female' 38.0 1 0]
 [3 'female' 26.0 0 0]
 ...
 [3 'female' nan 1 2]
 [1 'male' 26.0 0 0]
 [3 'male' 32.0 0 0]]


In [5]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan,strategy = 'mean')
imputer.fit(X[:,2:3])
X[:,2:3] = imputer.transform(X[:,2:3])

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan,strategy = 'mean')
imputer.fit(X[:,2:3])
X_test[:,2:3] = imputer.transform(X_test[:,2:3])

In [7]:
print(X)

[[3 'male' 22.0 1 0]
 [1 'female' 38.0 1 0]
 [3 'female' 26.0 0 0]
 ...
 [3 'female' 29.69911764705882 1 2]
 [1 'male' 26.0 0 0]
 [3 'male' 32.0 0 0]]


In [8]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X_test = np.array(ct.fit_transform(X_test))

In [10]:
print(X)

[[0.0 1.0 3 22.0 1 0]
 [1.0 0.0 1 38.0 1 0]
 [1.0 0.0 3 26.0 0 0]
 ...
 [1.0 0.0 3 29.69911764705882 1 2]
 [0.0 1.0 1 26.0 0 0]
 [0.0 1.0 3 32.0 0 0]]


In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X[:,2:6] = sc.fit_transform(X[:,2:6])
#no need to fit the test set
X_test[:,2:6] = sc.transform(X_test[:,2:6])

In [12]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X, y)

SVC(random_state=0)

In [13]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_test.reshape(len(y_test),1),y_pred.reshape(len(y_pred),1) ),1))

[[ 892    0]
 [ 893    1]
 [ 894    0]
 [ 895    0]
 [ 896    1]
 [ 897    0]
 [ 898    1]
 [ 899    0]
 [ 900    1]
 [ 901    0]
 [ 902    0]
 [ 903    0]
 [ 904    1]
 [ 905    0]
 [ 906    1]
 [ 907    1]
 [ 908    0]
 [ 909    0]
 [ 910    1]
 [ 911    1]
 [ 912    0]
 [ 913    1]
 [ 914    1]
 [ 915    0]
 [ 916    1]
 [ 917    0]
 [ 918    1]
 [ 919    0]
 [ 920    0]
 [ 921    0]
 [ 922    0]
 [ 923    0]
 [ 924    0]
 [ 925    0]
 [ 926    0]
 [ 927    0]
 [ 928    1]
 [ 929    1]
 [ 930    0]
 [ 931    0]
 [ 932    0]
 [ 933    0]
 [ 934    0]
 [ 935    1]
 [ 936    1]
 [ 937    0]
 [ 938    0]
 [ 939    0]
 [ 940    1]
 [ 941    1]
 [ 942    0]
 [ 943    0]
 [ 944    1]
 [ 945    1]
 [ 946    0]
 [ 947    0]
 [ 948    0]
 [ 949    0]
 [ 950    0]
 [ 951    1]
 [ 952    0]
 [ 953    0]
 [ 954    0]
 [ 955    1]
 [ 956    1]
 [ 957    1]
 [ 958    1]
 [ 959    0]
 [ 960    0]
 [ 961    0]
 [ 962    1]
 [ 963    0]
 [ 964    1]
 [ 965    0]
 [ 966    1]
 [ 967    0]
 [ 968    0]

In [14]:
submission = pd.DataFrame({
        "PassengerId": y_test,
        "Survived": y_pred
    })

In [15]:
submission = pd.DataFrame(submission).to_csv("output.csv",index = False)